In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [16]:
import pandas as pd
import numpy as np
import os
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib
import requests
from PIL import Image
import re
from io import BytesIO
import base64
import cv2
from PIL import Image, ImageEnhance, ImageFilter
import html5lib
import lxml
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from time import sleep
import random
from string import digits

In [3]:
def getFoodLinks(category, sub_category_link, class_directory):
    if category == "Foods and Nutrients":
        sub_category = category
    
    else:
        sub_category = re.search("_in_(.+?).html", sub_category_link).group(1)
    
    if sub_category in completed_subcategories:
        return None
    
    sub_category_links = []
    print(sub_category)
    for i in range(1,21):
        sub_category_links.append(sub_category_link.replace(".html", "") + "_page_" + str(i) + ".html")
    
    all_pages_links = []
    for link in sub_category_links:
        sleep(random.randint(1,3))
        driver.get(link)
        if "No matching foods found" in driver.page_source:
            break 
        try:
            page_foods = driver.find_elements_by_class_name(class_directory["food_item_link"])
            all_pages_links+=[food.get_attribute('href') for food in page_foods]
        except:
            sleep(20)
            break
            
    category_food_items_df = pd.DataFrame(all_pages_links, columns = ["Food_Item_Link"])
    category_food_items_df["Category"] = category
    category_food_items_df["Sub_Category"] = sub_category
    return category_food_items_df

In [100]:
def getAlternateData(food, homepage):
    #sleep(random.randint(2,4))
    
    driver.get(homepage)
    search_bar_input = driver.find_element_by_name("freetext")
    search_bar_input.click()
    search_bar_input.send_keys(food)
    search_btn = driver.find_element_by_class_name("searchNow")
    search_btn.click()
    #sleep(random.randint(2,4))

    search_results = driver.page_source
    soup = BeautifulSoup(search_results, "html.parser")
    food_link = soup.find("a", {"class": "list"}, href=True)
    
    try:
        food_website = nutrition_dataself_website + food_link["href"]
    except:
        food_details = {"Food Name": food, 
                        "Glycemic Load": np.NaN, 
                        "Completeness Score": np.NaN, 
                        "Amino Acid Score": np.NaN}
        
        return food_details

    driver.get(food_website)
    #sleep(random.randint(2,5))

    food_soup = driver.page_source
    food_soup = BeautifulSoup(food_soup, "html.parser")
    glycemic_load = food_soup.find("span", {"id": "SCORE_ESTIMATED_GLYCEMIC_LOAD"}).text
    food_scores = food_soup.find_all("div", {"class": "box_PQI"})
    food_scores = [score.text.strip() for score in food_scores]
    completeness_score, amino_acid_score = food_scores[0], food_scores[1]
    
    food_details = {"Food Name": food, 
                    "Glycemic Load": glycemic_load, 
                    "Completeness Score": completeness_score, 
                    "Amino Acid Score": amino_acid_score}
    
    return food_details

In [5]:
chromedriver_path = r"D:\Personal Projects\IP Data Extraction\chromedriver.exe"
nutrition_value_website = "https://www.nutritionvalue.org/"
nutrition_dataself_website = "https://nutritiondata.self.com/"
output_path = r"D:\College Work\DSCI 558 - Knowledge Graphs\Project\Output\Scraped Data\\"

#### Nutrition Value Extraction

In [6]:
food_categories_class = "l"
food_item_link = "table_item_name"
class_directory = {"food_categories_class": food_categories_class,
                   "food_item_link": food_item_link}

In [7]:
food_category_xpath = '//*[@id="main"]/tbody/tr[24]'
xpath_directory = {"food_category_xpath": food_category_xpath}

In [127]:
driver = webdriver.Chrome(executable_path=chromedriver_path)
driver.get(nutrition_dataself_website)

In [65]:
food_category_links = driver.find_element_by_xpath(xpath_directory["food_category_xpath"])
food_category_links = food_category_links.find_elements_by_tag_name("a")
food_category_links = [element.get_attribute('href') for element in food_category_links]

In [70]:
food_category_links_dict = {"Standard Reference Foods": food_category_links[0],
                            "Foundation Foods": food_category_links[1],
                            "Foods and Nutrients": food_category_links[2],
                            "Branded Foods": food_category_links[3]}

In [151]:
food_item_category_links = {}
for category in food_category_links_dict.keys():
    link = food_category_links_dict[category]
    if "categories" in link:
        driver.get(link)
        standard_food_links = driver.find_elements_by_class_name(class_directory["food_categories_class"])
        standard_food_links = [item.get_attribute('href') for item in standard_food_links]
        food_item_category_links[category] = standard_food_links
    else:
        food_item_category_links[category] = food_category_links_dict[category]

In [176]:
completed_categories = ["Standard Reference Foods", "Foundation Foods", "Foods and Nutrients"]

In [178]:
category_foods_df = pd.DataFrame()
for category in food_item_category_links.keys():
    if category in completed_categories:
        continue
    print("Category is:")
    print(category)
    if category == "Foods and Nutrients":
        sub_category = food_item_category_links[category]
        category_foods_df = category_foods_df.append(getFoodLinks(category, sub_category, class_directory), ignore_index = True)
    else:
        for sub_category in food_item_category_links[category]:
            category_foods_df = category_foods_df.append(getFoodLinks(category, sub_category, class_directory), ignore_index = True)

Category is:
Branded Foods
Dough_Based_Products_~_Meals_-_Not_Ready_to_Eat_-_Savoury_%28Frozen%29
Baking~Cooking_Mixes~Supplies_Variety_Packs
Flour_-_Cereal~Pulse_%28Shelf_Stable%29
Sauces_-_Cooking_%28Shelf_Stable%29
Cakes_-_Sweet_%28Shelf_Stable%29
Fruit_AND_Vegetable_Juice%2C_Nectars_AND_Fruit_Drinks
French_Fries%2C_Potatoes_AND_Onion_Rings
Frozen_Breakfast_Sandwiches%2C_Biscuits_AND_Meals
Eggs_AND_Egg_Substitutes
Frozen_Bread_AND_Dough
Deli_Salads
Sushi
Breakfast_Sandwiches%2C_Biscuits_AND_Meals
Plant_Based_Milk
Baking_Accessories
Alcohol
Other_Frozen_Meats
Lunch_Snacks_AND_Combinations
Prepared_Subs_AND_Sandwiches
Iced_AND_Bottle_Tea
Energy%2C_Protein_AND_Muscle_Recovery_Drinks
Liquid_Water_Enhancer
Other_Drinks
Soda
Baking
Grains
Breakfast_Foods
Specialty_Formula_Supplements
Green_Supplements
Processed_Cheese_AND_Cheese_Novelties
Weight_Control
Vitamins
Plant_Based_Water
Meal_Replacement_Supplements
Water
Sport_Drinks
Digestive_AND_Fiber_Supplements
Fruits%2C_Vegetables_AND_Produ

In [179]:
completed_crawling_df = completed_crawling_df.append(category_foods_df, ignore_index = True)

In [181]:
os.chdir(output_path)
completed_crawling_df.to_csv("Complete Extracted Data.csv", index = False)

#### Nutrition Data Self

In [79]:
extracted_foods = os.listdir(output_path)
extracted_foods = [food.replace(".csv", "") for food in extracted_foods]
extracted_foods = [food.replace("_", " ") for food in extracted_foods]
extracted_foods = [food.replace("(", "") for food in extracted_foods]
extracted_foods = [food.replace(")", "") for food in extracted_foods]
extracted_foods = [re.sub("[\(\[].*?[\)\]]", "", food) for food in extracted_foods]
extracted_foods = [food for food in extracted_foods if food!="Final Aggregated Data"]
remove_digits = str.maketrans('', '', digits)
extracted_foods = [food.lower() for food in extracted_foods]
extracted_foods = [food.translate(remove_digits) for food in extracted_foods]
extracted_foods = [food.split("by")[0] for food in extracted_foods]
extracted_foods = [food.split() for food in extracted_foods]
extracted_foods = [" ".join((sorted(set(food), key=food.index))) for food in extracted_foods]
extracted_foods = [food.replace("  ", " ") for food in extracted_foods]
extracted_foods = [food.strip() for food in extracted_foods]
extracted_foods = list(set(extracted_foods))
extracted_foods = [food for food in extracted_foods if food!=""]
extracted_foods = [food.replace(" oz", "") for food in extracted_foods]

In [80]:
final_extracted_foods = []
for food in extracted_foods:
    if food.split()[-1]=='and':
        food = food.split()[:-1]
        final_extracted_foods.append(" ".join(food))
    else:
        final_extracted_foods.append(food)

final_extracted_foods = [food.split()[:5] for food in final_extracted_foods]
final_extracted_foods = [" ".join(food) for food in final_extracted_foods]

In [136]:
alternate_df = pd.DataFrame()
for i in range(len(final_extracted_foods)):
    food = final_extracted_foods[i]
    print(i)
    print(food)
    
    alternate_df = alternate_df.append(getAlternateData(food, nutrition_dataself_website), ignore_index = True)

0
campbells chunky soup potato ham
1
bacnvacregfloz
2
abbott eas advant edge pure
3
betty crocker walnut chocolate chip
4
chicken raw all classes heart
5
month aged cheddar shredded natural
6
homestyle pretzel rub for pork
7
organic coconut water
8
calrose medium grain rice
9
dryden palmer barista crystal sticks
10
lamb braised cooked separable lean
11
campbells slow kettle soup cheese
12
chicken meat and skin drumstick
13
diet mtn dew liter plastic
14
carrots solids and liquids regular
15
chocolate milk made from no
16
doritos party size cool ranch
17
cranberry cosmos cocktail rimming sugar
18
egg and sausage on bun
19
perch no added fat fried
20
swiss miss hot cocoa mix
21
mojito mix fine cocktail mixtures
22
beef grilled cooked choice trimmed
23
pork unheated separable lean and
24
cookie not further speficied
25
cocktail sauce with horseradish
26
hard white wheat
27
cactus fat added cooked
28
rhubarb leaves alaska native wild
29
pork panbroil heated separable lean
30
applesauce unsw

247
creamy milk chocolate flavored original
248
bittersweet orange marmalade preserves
249
greens no added fat cooked
250
mothbeans without salt boiled cooked
251
coffee reconstituted prelightened and presweetened
252
cereals readytoeat raisin bran cereal
253
pears raw
254
doughnut custardfilled
255
american finest sandwich slices
256
barbeque sauce honey
257
balsamic vinaigrette aged vinegar honey
258
lemon lime bubbles sparkling probiotic
259
burdock cooked
260
complete plant protein chocolate
261
fruit punch lower sugar thirst
262
cereal frosted oat with marshmallows
263
cookie with icing or filling
264
all natural peach ice cream
265
rice fat added with dark
266
a hint of mango hot
267
dulce de leche
268
busseto foods salami collection
269
green way flavor enhancer raspberry
270
huberts lemonade watermelon glass bottle
271
act ii extreme butter
272
tyson grilled ready seasoned steak
273
whiting made with butter fried
274
beans low sodium solids and
275
coconut water sports drink
27

499
good sense white quinoa
500
artificially flavored soda
501
bologna made with turkey and
502
americas sweet salsa
503
fry mix beer batter
504
infant formula prepared from powder
505
coffee creamer flavored sugar free
506
rabbit stew with potatoes and
507
fish drained solids canned in
508
salad dressing without salt regular
509
assorted flavors freezer pops
510
barilla whole wheat pasta
511
marie callenders mac and cheese
512
chicken thigh from fast food
513
cereal or granola bar with
514
espresso forte coffee
515
pineapple solids and liquids heavy
516
fiber one calorie orange cranberry
517
fruit pretzel braids monterey jack
518
restaurant tripe soup latino
519
ensure high protein shake homemade
520
butternut sage
521
pasta restaurant and added vegetables
522
green beans no added fat
523
essential everyday sloppy joe original
524
blueberry lemonade sparkling water
525
beverages powder high protein nutritional
526
jelly all flavors reduced sugar
527
lamb raw tongue swiss cut
528
a cla

744
chicory beverage
745
beef not specified as to
746
beef braised cooked all grades
747
restaurant onion rings family style
748
potato with vegetables peel not
749
oil for woks and light
750
annies organic berry patch strawberry
751
totinos combination pizza rolls ct
752
harmless harvest namacha cold brewed
753
burrito with chicken and sour
754
syrups reducedcalorie pancake table blends
755
yogurt fruit not specified as
756
classico vodka pasta sauce
757
ahold flavored seltzer water watermelon
758
keebler town house crackers italian
759
cookie brownie butterscotch
760
tropicana trop raspberry lemonade juice
761
nutritional powder mix eas soy
762
opossum cooked
763
black forest turkey breast and
764
medium spicy beer salt
765
dailys precooked bacon special readytofinish
766
strudel berry
767
coconut cream sweetened canned
768
snacks mixed flavors crunchy kashi
769
lima beans with salt drained
770
all natural sunflower oil
771
buffalo style chicken snacker
772
fat free premium deli oven

990
basics chickpeas
991
split pea soup prepared with
992
mustard greens fat added cooked
993
pork soybased sauce and darkgreen
994
burrito or quesadilla with egg
995
totinos pizza rolls brnd snacks
996
amys premium organic pasta sauce
997
orville ultra butter pop up
998
caffeine free punch soda
999
chicken breader
1000
oka coco coconut water watermelon
1001
sausage not further speficied
1002
authentic italian soda
1003
beverages powder chocolate malted drink
1004
lamb broiled cooked trimmed to
1005
bb soy sauce
1006
orange juice raw
1007
carb control shakes
1008
sesame chicken
1009
swiss miss hot cocoa mix
1010
classic pizza supreme
1011
blueberry toast crunch cereal
1012
ricearoni unprepared chicken flavor
1013
liquid infusions for cream orange
1014
cod made with oil baked
1015
whole wheat cereal fat added
1016
amatirciana sauce
1017
grapefruit juice with calcium added
1018
black cherry premium preserves
1019
pork roasted extra lean approximately
1020
fish liver alaska native broad
1

...

3116
clams raw
3117
mustard greens with salt drained
3118
classic alfredo cheese creations sauce
3119
beef raw all grades trimmed
3120
gelatin desserts prepared with water
3121
pheasant total edible cooked
3122
agrisicilia sicilian lemon marmalade
3123
caramel cappuccino coffee protein beverage
3124
la choy chow mein noodles
3125
berry zero sugar thirst quencher
3126
alessi eggplant appetizer
3127
chesters fries flamin hot ounce
3128
incaparina unprepared dry mix corn
3129
ahold thin sliced honey ham
3130
crushed italian tomatoes
3131
cereals dry raisin and spice
3132
turkey raw meat and skin
3133
puddings prepared with milk regular
3134
bourbon chicken sauce
3135
onions tops only young green
3136
enchilada with chicken redchile or
3137
pork roasted cooked separable lean
3138
chicken skin eaten stewed not
3139
cold pressed juice basil berry
3140
chokecherries pitted northern plains indians
3141
bushs best southwest black bean
3142
claim jumper key lime pie
3143
beet greens with salt dr

3354
soft taco with meat from
3355
banana quick bread mix
3356
cheese italian low moisture partskim
3357
marinara sauce
3358
corn no salt added cream
3359
french toast reduced fat whole
3360
cascade ice sparkling water with
3361
pappalos uncured pepperoni pizza snacks
3362
naked pure fruit mighty mango
3363
regular small curd cottage cheese
3364
sausage raw fresh turkey
3365
an enriched macaroni product penne
3366
aged asiago
3367
chocolate flavored protein nutrition shake
3368
waffles readytoheat frozen chocolate chip
3369
ben jerrys ice cream smores
3370
beef andor darkgreen leafy tomatobased
3371
crisp thin crackers sea salt
3372
cabbage no added fat cooked
3373
sweetened iced arabica soluble coffee
3374
papa johns cheese pizza original
3375
snacks airpopped popcorn
3376
tropicana twister light fruit punch
3377
bkribpklnrcmeatysklsibpcz
3378
infant formula not reconstituted with
3379
beef raw choice trimmed to
3380
antioxidant rich cinnamon honey water
3381
salmon cream cheese sushi

3595
decaffeinated green tea
3596
congee and vegetables andor seafood
3597
medium fiery sweet sauce
3598
shark not specified as to
3599
natural double cheddar cheese dried
3600
gravy dry instant beef
3601
vegetable and turkey strained ba
3602
potato not specified as to
3603
chia water
3604
milk with added vitamin a
3605
fish raw sheepshead
3606
burrito sauce mix
3607
brick oven crust cheesetrio pizza
3608
x concentrate cold brew coffee
3609
fried chili in oil
3610
barley with mediterranean herbs and
3611
mustard greens no added fat
3612
fish total can contents canned
3613
light brown premium pure cane
3614
keebler murray sugar free cookies
3615
california extra virgin olive oil
3616
plsbry tstr strdl pastries lmtd
3617
odoms tennessee pride sausage gravy
3618
arugula basil pesto
3619
gatorade g mixed berry thirst
3620
santiam mixed vegetableoz
3621
ancient grains breadcrumbs
3622
all natural ultra premium ice
3623
pina colada cocktail mix
3624
beef panfried cooked choice trimmed
3625
b

3835
campbells chunky soup jambalaya
3836
cereal general mills lucky charms
3837
hard candy
3838
milled flax seed
3839
pork roasted cooked separable lean
3840
energy drink enhancer
3841
beef cooked all grades trimmed
3842
coconut water beverage vanilla chai
3843
waffle from frozen whole grain
3844
pork unheated canned extra lean
3845
calnaturale svelte organic protein shake
3846
orange performance sports drink
3847
egg substitute with cheese and
3848
fleischmanns volwhip frozen egg whites
3849
pork raw belly fresh
3850
dark roast lightning coffee pods
3851
gardenburger gb veggie burgers black
3852
pringles crisps buffalo ranch
3853
ham puerto rican style and
3854
beef raw choice trimmed to
3855
fish raw farmed rainbow trout
3856
noodles dry somen japanese
3857
asparagus gruyere ravioli
3858
beef raw all grades trimmed
3859
fast foods prepared with garlic
3860
pie peach
3861
real bacon bits smoke
3862
radishes without salt drained boiled
3863
chicken tikka cooking sauce for
3864
italian

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: Unable to receive message from renderer)
  (Session info: chrome=107.0.5304.107)


In [137]:
alternate_df.shape

(3950, 4)

In [138]:
alternate_df.to_csv("Sample 9 - Nutrition Data Self.csv", index = False)

In [141]:
alternate_df_final = pd.DataFrame()
for i in range(2,10):
    alternate_df_temp = pd.read_csv("Sample "+ str(i) + " - Nutrition Data Self.csv")
    alternate_df_final = alternate_df_final.append(alternate_df_temp, ignore_index = True)

In [144]:
alternate_df_final.to_csv("Nutrition Data Self Data.csv", index = False)